# FINAL EXAM OF ECON 5821


#### 1155198448 PAN, Qiying

#### 1155198430 MA, Xinlan 

#### 1155207188 Huang, Lisha 

In our final exam, we will go though it the following procedure
* data process
* AR(1)
* LASSO & RIDGE
* Random Forest
* Gredient Boosting

In [37]:
# installing the package we need in this program
if (!require("readxl")) install.packages("readxl")
library(readxl)
if (!require("glmnet")) install.packages("glmnet")
library(glmnet)
if (!require("caret")) install.packages("caret")
library(readxl)
if (!require("tidyverse")) install.packages("tidyverse")
library(readxl)
if (!require("doParallel")) install.packages("doParallel")
library(doParallel)
if (!require("foreach")) install.packages("foreach")
library(foreach)
if (!require("gbm")) install.packages("gbm")
library(gbm)
if (!require("forecast")) install.packages("forecast")
library(forecast)
if (!require("tseries")) install.packages("tseries")
library(tseries)
if (!require("ranger")) install.packages("ranger")
library(ranger)

In [38]:
cl <- makeCluster(16)  # Create a parallel cluster with 16 cores
registerDoParallel(cl) 

In [39]:
# process the data
data_train <- read_excel("US_PCE_training.xlsx")
# delet the NA value 
data_train = na.omit(data_train)
# delet the row of 'month'
data_train = data_train[data_train$month != 'month',]
data_train = t(data_train)

#  rename the variable to consist with 'xlsx' file
colnames(data_train) <- unlist(data_train[1, ])
#delet the first row, it the variable name of column
data_train <- data_train[-1, ] 

#head(data_train) 

In [40]:
# process the data.   the same procedure of above cell
data_score <- read_excel("US_PCE_testing_fake.xlsx")
data_score = na.omit(data_score)
data_score = data_score[data_score$month != 'month',]
data_score = t(data_score)
colnames(data_score) <- unlist(data_score[1, ])
data_score <- data_score[-1, ] 

#head(data_score)  

In [41]:
#check if  NA exist.
na_coords_train <- which(is.na(data_train), arr.ind = TRUE)
na_coords_score <- which(is.na(data_score), arr.ind = TRUE)
print(na_coords_train)
print(na_coords_score)

     row col
     row col


In [42]:
# Convert the data to dataframe format
data_train = as.data.frame(data_train)
data_score = as.data.frame(data_score)
# Convert the data to numeric
data_train[] <- lapply(data_train, as.numeric)
data_score[] <- lapply(data_score, as.numeric)

In [43]:
# creat copys
data_train_copy = data_train
data_score_copy = data_score

In [44]:
# 将PCE处理成通胀
data_train = data_train_copy
data_score = data_score_copy 

data_score = rbind(tail(data_train,1),data_score)

#计算inflation
data_train$inflation = (log(data_train$`Personal consumption expenditures`)-log(lag(data_train$`Personal consumption expenditures`)))*12
data_score$inflation = (log(data_score$`Personal consumption expenditures`)-log(lag(data_score$`Personal consumption expenditures`)))*12
#删除原变量
data_train = data_train[, !names(data_train) %in% "Personal consumption expenditures"]
data_score = data_score[, !names(data_score) %in% "Personal consumption expenditures"]
#移动最后一列到第一列
data_train <- data_train[, c(ncol(data_train), 1:(ncol(data_train)-1))]
data_score <- data_score[, c(ncol(data_score), 1:(ncol(data_score)-1))]
#删除第一个空观测值 
data_train$yt = data_train$inflation
data_score$yt = data_score$inflation
data_score = data_score[-1,]
head(data_train)
#head(data_score)
#tail(data_train)
#tail(data_score)

,inflation,New domestic autos,New foreign autos,New light trucks,Net transactions in used autos,Used auto margin,Employee reimbursement,Used light trucks,Tires,Accessories and parts,⋯,Nonprofit nursing homes services to households,Recreation services to households,Education services to households,Social services to households,Religious organizations' services to households.1,Foundations and grantmaking and giving services to households.1,Services of social advocacy establishments to households,Civic and social organizations' services to households,Professional advocacy services to households,yt
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,NA,37.337,37.348,32.438,22.544,5.006,11.596,25.124,46.107,27.263,⋯,6.219,9.916,2.964,13.044,12.152,10.908,12.304,13.859,8.770,NA
2,0.011864352,37.337,37.348,32.436,22.737,5.083,11.671,25.316,46.026,27.204,⋯,6.271,9.986,2.928,12.701,11.911,10.671,11.980,13.494,8.818,0.011864352
3,0.007903056,37.481,37.492,32.559,23.010,5.044,11.742,25.603,46.019,27.268,⋯,6.263,10.070,2.886,12.442,11.711,10.570,11.736,13.219,8.874,0.007903056
4,0.023677987,37.409,37.420,32.493,23.136,5.160,11.746,25.790,46.020,27.329,⋯,6.287,10.176,2.852,12.157,11.502,10.252,11.468,12.917,8.937,0.023677987
5,0.006306247,37.553,37.564,32.616,23.134,5.198,11.751,25.786,45.927,27.337,⋯,6.313,10.241,2.811,11.928,11.285,9.977,11.251,12.673,8.939,0.006306247
6,0.034625245,37.625,37.636,32.678,23.343,5.313,11.821,25.978,45.940,27.271,⋯,6.342,10.318,2.777,11.664,11.061,9.940,11.000,12.391,9.019,0.034625245


In [45]:
print(which(is.na(data_train), arr.ind = TRUE))

  row col
1   1   1
1   1 206


the following cells is to process the data that need for 1 month predition, 3 month predition and 12 month prediction<br>Sorry for not streamlining the code by defining functions, which resulted in a lot of repetition in the subsequent parts. 

In [46]:
#data preparation for 1 month prediction
data_train_1 = data_train
data_score_1 = rbind(tail(data_train_1,1),data_score)   

data_train_1$inflation = lead(data_train_1$inflation,1) 
data_score_1$inflation = lead(data_score_1$inflation,1) 

data_train_1 <- data_train_1[1:(nrow(data_train_1) - 1), ]
data_score_1 <- data_score_1[1:(nrow(data_score_1) - 1), ]
data_train_1 = data_train_1[-1,]
#head(data_train_1)
#head(data_score_1)
#tail(data_train_1)
#tail(data_score_1)


this excel simply shows the processed data for one month prediction
| Dependent Variable Y| Independent variable X1 |
|---------|---------|
| \$Y_{t+1}\$ | \$X_t\$ | 
| \$Y_{t+2}\$ | \$X_{t+1}\$ |  

where  \$Y_{t+1}\$ stand for inflation in period t+1.

In [59]:
# data preparation for 3 month prediction.    Duplicated
data_train_3 = data_train
data_score_3 = rbind(tail(data_train_3,3),data_score)

data_train_3$inflation = lead(data_train_3$inflation,3) 
data_score_3$inflation = lead(data_score_3$inflation,3) 

data_train_3 <- data_train_3[1:(nrow(data_train_3) - 3), ]
data_score_3 <- data_score_3[1:(nrow(data_score_3) - 3), ]
data_train_3 = data_train_3[-1,]
head(data_train_3)
#head(data_score_3)
#tail(data_train_3)
#tail(data_score_3)

,inflation,New domestic autos,New foreign autos,New light trucks,Net transactions in used autos,Used auto margin,Employee reimbursement,Used light trucks,Tires,Accessories and parts,⋯,Nonprofit nursing homes services to households,Recreation services to households,Education services to households,Social services to households,Religious organizations' services to households.1,Foundations and grantmaking and giving services to households.1,Services of social advocacy establishments to households,Civic and social organizations' services to households,Professional advocacy services to households,yt
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2,0.006306247,37.337,37.348,32.436,22.737,5.083,11.671,25.316,46.026,27.204,⋯,6.271,9.986,2.928,12.701,11.911,10.671,11.980,13.494,8.818,0.011864352
3,0.034625245,37.481,37.492,32.559,23.010,5.044,11.742,25.603,46.019,27.268,⋯,6.263,10.070,2.886,12.442,11.711,10.570,11.736,13.219,8.874,0.007903056
4,0.025119392,37.409,37.420,32.493,23.136,5.160,11.746,25.790,46.020,27.329,⋯,6.287,10.176,2.852,12.157,11.502,10.252,11.468,12.917,8.937,0.023677987
5,0.017239131,37.553,37.564,32.616,23.134,5.198,11.751,25.786,45.927,27.337,⋯,6.313,10.241,2.811,11.928,11.285,9.977,11.251,12.673,8.939,0.006306247
6,0.031280565,37.625,37.636,32.678,23.343,5.313,11.821,25.978,45.940,27.271,⋯,6.342,10.318,2.777,11.664,11.061,9.940,11.000,12.391,9.019,0.034625245
7,0.028862106,37.625,37.636,32.678,23.811,5.352,11.813,26.463,46.110,26.932,⋯,6.373,10.407,2.781,11.665,11.141,9.917,10.998,12.388,9.095,0.025119392


In [63]:
# data preparation for 12 month prediction.    Duplicated
data_train_12 = data_train
data_score_12 = rbind(tail(data_train_12,12),data_score)

data_train_12$inflation = lead(data_train_12$inflation,12) 
data_score_12$inflation = lead(data_score_12$inflation,12) 

data_train_12 <- data_train_12[1:(nrow(data_train_12) - 12), ]
data_score_12 <- data_score_12[1:(nrow(data_score_12) - 12), ]
data_train_12 = data_train_12[-1,]
#head(data_train_12)
#head(data_score_12)
#tail(data_train_12)
#tail(data_score_12)

the following cells firstlly seperate the data to 70% train data and 30% test data.  then we also seperate the Y and X.<br> 
train 1 stand for 70% train data, test stand for 30% test data. score stand for real data post on May 16 

In [64]:
# train data and test data preparation     1 month predition
set.seed(123)  
train_size_1 <- floor(0.7 * nrow(data_train_1))

# Because it's time-series data, we split the training and testing sets directly based on sequence.
train_data_1 <- data_train_1[1:train_size_1, ]
test_data_1 <- data_train_1[(train_size_1 + 1):nrow(data_train_1), ]


train_x_1 <- data.matrix(train_data_1[,-1])  
train_y_1 <- train_data_1[,1]

test_x_1 <- data.matrix(test_data_1[,-1])
test_y_1 <- test_data_1[,1]

score_x_1 <- data.matrix(data_score_1[,-1])
score_y_1 <- data_score_1[,1]

In [65]:
# train data and test data preparation     3 month predition
train_size_3 <- floor(0.7 * nrow(data_train_3))

# Because it's time-series data, we split the training and testing sets directly based on sequence.
train_data_3 <- data_train_3[1:train_size_3, ]
test_data_3 <- data_train_3[(train_size_3 + 1):nrow(data_train_3), ]


train_x_3 <- data.matrix(train_data_3[,-1])  
train_y_3 <- train_data_3[,1]

test_x_3 <- data.matrix(test_data_3[,-1])
test_y_3 <- test_data_3[,1]

score_x_3 <- data.matrix(data_score_3[,-1])
score_y_3 <- data_score_3[,1]

In [66]:
# train data and test data preparation     12 month predition
train_size_12 <- floor(0.7 * nrow(data_train_12))

# Because it's time-series data, we split the training and testing sets directly based on sequence.
train_data_12 <- data_train_12[1:train_size_12, ]
test_data_12 <- data_train_12[(train_size_12 + 1):nrow(data_train_12), ]


train_x_12 <- data.matrix(train_data_12[,-1])  
train_y_12 <- train_data_12[,1]

test_x_12 <- data.matrix(test_data_12[,-1])
test_y_12 <- test_data_12[,1]

score_x_12 <- data.matrix(data_score_12[,-1])
score_y_12 <- data_score_12[,1]

# AR(1)

the following code show AR(1) model of 1 month predition, 3 month predition and 12 month predition.<br>
we simply use OLS method to estimate this model.
$$Y_t = c + \phi_1 Y_{t-1} + \epsilon_t$$in order to compare with the other meshine learning methods, we only use 70% train data to estimate this model. then we use this "trained" model to predict the test data.


In [52]:
# 1 month prediction of AR(1)
train_y = train_y_1
test_y = test_y_1
score_y = score_y_1

# train the AR(1) model using 70% train data
ar_model = lm(train_y[-1] ~ lag(train_y)[-1])


# train mse
prediction = numeric(length(lag(train_y)[-1]))   #Initialize a list to store each predicted value yt
train_y_ar = lag(train_y)[-1]
for (i in 1:length(lag(train_y)[-1])) {
  prediction[i] = coef(ar_model)[1]+coef(ar_model)[2]*train_y_ar[i]
}

MSE = mean((prediction - train_y[-1])^2) 
print(paste("MSE of the 70% train data (one month prediction): ", MSE))

# test mse
prediction = numeric(length(test_y))   #Initialize a list to store each predicted value yt
test_y_ar = c(tail(train_y,1),test_y[-length(test_y)])
for (i in 1:length(test_y)) {
  prediction[i] = coef(ar_model)[1]+coef(ar_model)[2]*test_y_ar[i]
}

MSE = mean((prediction - test_y)^2) # MSE of test data
print(paste("MSE of the rest 30% data (one month prediction): ", MSE))


#score mse
prediction_score = numeric(length(score_y))   # MSE of score data
score_y_ar = c(tail(test_y,1),score_y[-length(score_y)])
for (i in 1:length(score_y)) {
  prediction_score[i] = coef(ar_model)[1]+coef(ar_model)[2]*score_y_ar[i]
}

MSE = mean((prediction_score - score_y)^2)
print(paste("MSE of the real test data (one month prediction): ", MSE))



[1] "MSE of the 70% train data (one month prediction):  0.000378794438041514"
[1] "MSE of the rest 30% data (one month prediction):  0.000577229495917976"
[1] "MSE of the real test data (one month prediction):  78.2171299183431"


In [53]:
# 3 month prediction of AR(1)
#NOTE:  AR(1) only use dependent value, thus we let  'train_y = train_y_1 '   and then implement "lag(train_y,3)" function to estimate 3 month predition AR(1)
train_y = train_y_1
test_y = test_y_1
score_y = score_y_1

# train the AR(1) model using 70% train data
ar_model = lm(train_y[-(1:3)] ~ lag(train_y,3)[-(1:3)])


# train mse
prediction = numeric(length(lag(train_y,3)[-(1:3)]))   #Initialize a list to store each predicted value yt
train_y_ar = lag(train_y,3)[-(1:3)]
for (i in 1:length(lag(train_y,3)[-(1:3)])) {
  prediction[i] = coef(ar_model)[1]+coef(ar_model)[2]*train_y_ar[i]
}

MSE = mean((prediction - train_y[-(1:3)])^2) 
print(paste("MSE of the 70% train data (three month prediction): ", MSE))


# test mse
prediction = numeric(length(test_y))
test_y_ar = c(tail(train_y,3),test_y[1:(length(test_y)-3)])
for (i in 1:length(test_y)) {
  prediction[i] = coef(ar_model)[1]+coef(ar_model)[2]*test_y_ar[i]
}

MSE = mean((prediction - test_y)^2)
print(paste("MSE of the rest 30% data (three month prediction): ", MSE))


#score mse
prediction_score = numeric(length(score_y))
score_y_ar = c(tail(test_y,3),score_y[1:(length(score_y)-3)])
for (i in 1:length(score_y)) {
  prediction_score[i] = coef(ar_model)[1]+coef(ar_model)[2]*score_y_ar[i]
}

MSE = mean((prediction_score - score_y)^2)
print(paste("MSE of the real test data (three month prediction): ", MSE))

[1] "MSE of the 70% train data (three month prediction):  0.000488677768046148"
[1] "MSE of the rest 30% data (three month prediction):  0.000957063522439213"
[1] "MSE of the real test data (three month prediction):  49.2549575012447"


In [54]:
# 12 month prediction of AR(1)
#NOTE:  AR(1) only use dependent value, thus we let  'train_y = train_y_1 '   and then implement "lag(train_y,12)" function to estimate 12 month predition AR(1)
train_y = train_y_1
test_y = test_y_1
score_y = score_y_1

# train the AR(1) model using 70% train data
ar_model = lm(train_y[-(1:12)] ~ lag(train_y,12)[-(1:12)])


# train mse
prediction = numeric(length(lag(train_y,12)[-(1:12)]))   #Initialize a list to store each predicted value yt
train_y_ar = lag(train_y,12)[-(1:12)]
for (i in 1:length(lag(train_y,12)[-(1:12)])) {
  prediction[i] = coef(ar_model)[1]+coef(ar_model)[2]*train_y_ar[i]
}

MSE = mean((prediction - train_y[-(1:12)])^2) 
print(paste("MSE of the 70% train data (12 month prediction): ", MSE))

# test mse
prediction = numeric(length(test_y))
test_y_ar = c(tail(train_y,12),test_y[1:(length(test_y)-12)])
for (i in 1:length(test_y)) {
  prediction[i] = coef(ar_model)[1]+coef(ar_model)[2]*test_y_ar[i]
}

MSE = mean((prediction - test_y)^2)
print(paste("MSE of the rest 30% data (12 month prediction): ", MSE))



#score mse
prediction_score = numeric(length(score_y))
score_y_ar = c(tail(test_y,12),score_y[1:(length(score_y)-12)])
for (i in 1:length(score_y)) {
  prediction_score[i] = coef(ar_model)[1]+coef(ar_model)[2]*score_y_ar[i]
}

MSE = mean((prediction_score - score_y)^2)
print(paste("MSE of the real test data (12 month prediction): ", MSE))

[1] "MSE of the 70% train data (12 month prediction):  0.000574751635870192"
[1] "MSE of the rest 30% data (12 month prediction):  0.00101029907637994"
[1] "MSE of the real test data (12 month prediction):  42.0167810877226"


## Lasso and Ridge

in this part, we use caret package to help conduct CV for lasso and ridge model. Still,we use 70% train data to train Lasso and Ridge regression model. <br>firstly, we apply CV method to choose best lambda, and use the best lambda to train lasso(Ridge) model. since the data is time series data, we use Sliding Window Cross-Validation<br>then use the trained model to predict test data and score data

In [55]:
# ridge and lasso  model for 1 month prediction
train_x = train_x_1
train_y = train_y_1
test_x = test_x_1
test_y = test_y_1
score_x=score_x_1
score_y = score_y_1

train_data = train_data_1
test_data = test_data_1
data_scoreMSE = data_score_1

set.seed(1)
# CV to choose best lambda.     Ridge

# Sliding Window Cross-Validation
train_control <- trainControl(method = "timeslice",
                              initialWindow = floor(0.7*nrow(train_x)),
                              horizon = 12,
                              skip = 3,
                              fixedWindow = TRUE,
                              allowParallel = TRUE)

#choose best lambda, alpha = 0 is ridge regression
tuneGrid <- expand.grid(
  alpha = 0,
  lambda = 10^seq(5, -5, length=10)
) 


model <- train(train_x, train_y,
               method = "glmnet",
               tuneGrid = tuneGrid,
               trControl = train_control);




ridge.mod = glmnet(train_x, train_y, alpha=0, lambda=model$bestTune$lambda)

# train mse
ridge.pred <- predict(ridge.mod, newx = train_x )
mse <- mean((ridge.pred - train_data[, 1])^2)
print(paste("MSE of the 70% train data (1 month prediction): ", mse)) 

# test mse
ridge.pred <- predict(ridge.mod, newx = test_x )
mse <- mean((ridge.pred - test_data[, 1])^2)
print(paste("MSE of the rest 30% data (1 month prediction): ", mse)) 

# score mse
ridge.pred <- predict(ridge.mod,  newx = score_x )
mse_score <- mean((ridge.pred - data_scoreMSE[, 1])^2)
print(paste("MSE of real test data (1 month prediction): ", mse_score)) 

Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
“There were missing values in resampled performance measures.”


[1] "MSE of the 70% train data (1 month prediction):  0.000341170935698251"
[1] "MSE of the rest 30% data (1 month prediction):  0.000876712948271478"
[1] "MSE of real test data (1 month prediction):  43.6374984471829"


In [56]:
# CV to choose best lambda.     Lasso

# Sliding Window Cross-Validation
train_control <- trainControl(method = "timeslice",
                              initialWindow = floor(0.7*nrow(train_x)),
                              horizon = 1,
                              skip = 1,
                              fixedWindow = TRUE,
                              allowParallel = TRUE)

#choose best lambda, alpha = 1 is lasso regression
tuneGrid <- expand.grid(
  alpha = 1,
  lambda = 10^seq(2, -5, length=50)
) 


model <- train(train_x, train_y,
               method = "glmnet",
               tuneGrid = tuneGrid,
               trControl = train_control);


lasso.mod = glmnet(train_x, train_y, alpha=1, lambda=model$bestTune$lambda)

# train mse
lasso.pred <- predict(lasso.mod, newx = train_x )
mse <- mean((lasso.pred - train_data[, 1])^2)
print(paste("MSE of the rest 70% train data (1 month prediction): ", mse)) 

# test mse
lasso.pred <- predict(lasso.mod, newx = test_x )
mse <- mean((lasso.pred - test_data[, 1])^2)
print(paste("MSE of the rest 30% data (1 month prediction): ", mse)) 

# score mse
lasso.pred <- predict(lasso.mod,  newx = score_x )
mse_score <- mean((lasso.pred - data_scoreMSE[, 1])^2)
print(paste("MSE of real test data (1 month prediction): ", mse_score)) 

Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
“There were missing values in resampled performance measures.”


[1] "MSE of the rest 70% train data (1 month prediction):  0.000312779235611123"
[1] "MSE of the rest 30% data (1 month prediction):  0.000847007023392567"
[1] "MSE of real test data (1 month prediction):  56.4119188834437"


In [61]:
# ridge and lasso  model for 3 month prediction
train_x = train_x_3
train_y = train_y_3
test_x = test_x_3
test_y = test_y_3
score_x=score_x_3
score_y = score_y_3

train_data = train_data_3
test_data = test_data_3
data_scoreMSE = data_score_3


# CV to choose best lambda.     Ridge

# Sliding Window Cross-Validation
train_control <- trainControl(method = "timeslice",
                              initialWindow = floor(0.7*nrow(train_x)),
                              horizon = 1,
                              skip = 1,
                              fixedWindow = TRUE,
                              allowParallel = TRUE)

#choose best lambda
tuneGrid <- expand.grid(
  alpha = 0,   #ridge
  lambda = 10^seq(2, -5, length=50)
) 


model <- train(train_x, train_y,
               method = "glmnet",
               tuneGrid = tuneGrid,
               trControl = train_control);



ridge.mod = glmnet(train_x, train_y, alpha=0, lambda=model$bestTune$lambda)

# train mse
ridge.pred <- predict(ridge.mod, newx = train_x )
mse <- mean((ridge.pred - train_data[, 1])^2)
print(paste("MSE of the 70% train data (3 month prediction): ", mse)) 


# test mse
ridge.pred <- predict(ridge.mod, newx = test_x )
mse <- mean((ridge.pred - test_data[, 1])^2)
print(paste("MSE of the rest 30% data (3 month prediction): ", mse)) 


# score mse
ridge.pred <- predict(ridge.mod,  newx = score_x )
mse_score <- mean((ridge.pred - data_scoreMSE[, 1])^2)
print(paste("MSE of real test data (3 month prediction): ", mse_score)) 

Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
“There were missing values in resampled performance measures.”


[1] "MSE of the 70% train data (3 month prediction):  0.000429040826316357"
[1] "MSE of the rest 30% data (3 month prediction):  0.000664351851957809"
[1] "MSE of real test data (3 month prediction):  33.7429132075958"


In [62]:
# CV to choose best lambda.     Lasso

# Sliding Window Cross-Validation
train_control <- trainControl(method = "timeslice",
                              initialWindow = floor(0.7*nrow(train_x)),
                              horizon = 1,
                              skip = 1,
                              fixedWindow = TRUE,
                              allowParallel = TRUE)

# choose best lambda
tuneGrid <- expand.grid(
  alpha = 1,  #lasso
  lambda = 10^seq(2, -5, length=100)
) 


model <- train(train_x, train_y,
               method = "glmnet",
               tuneGrid = tuneGrid,
               trControl = train_control);



lasso.mod = glmnet(train_x, train_y, alpha=1, lambda=model$bestTune$lambda)

# train mse
lasso.pred <- predict(lasso.mod, newx = train_x )
mse <- mean((lasso.pred - train_data[, 1])^2)
print(paste("MSE of the rest 70% train data (3 month prediction): ", mse)) 

# test mse
lasso.pred <- predict(lasso.mod, newx = test_x )
mse <- mean((lasso.pred - test_data[, 1])^2)
print(paste("MSE of the rest 30% data (3 month prediction): ", mse)) 

# test mse
lasso.pred <- predict(lasso.mod,  newx = score_x )
mse_score <- mean((lasso.pred - data_scoreMSE[, 1])^2)
print(paste("MSE of real test data (3 month prediction): ", mse_score)) 

Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
“There were missing values in resampled performance measures.”


[1] "MSE of the rest 70% train data (3 month prediction):  0.000397662311094666"
[1] "MSE of the rest 30% data (3 month prediction):  0.000953923847315587"
[1] "MSE of real test data (3 month prediction):  36.940760195847"


In [67]:
# ridge and lasso  model for 12 month prediction
train_x = train_x_12
train_y = train_y_12
test_x = test_x_12
test_y = test_y_12
score_x=score_x_12
score_y = score_y_12

train_data = train_data_12
test_data = test_data_12
data_scoreMSE = data_score_12


# CV to choose best lambda.    ridge

# Sliding Window Cross-Validation
train_control <- trainControl(method = "timeslice",
                              initialWindow = floor(0.7*nrow(train_x)),
                              horizon = 1,
                              skip = 1,
                              fixedWindow = TRUE,
                              allowParallel = TRUE)

#choose best lambda
tuneGrid <- expand.grid(
  alpha = 0,  #ridge
  lambda = 10^seq(2, -7, length=100)
) 


model <- train(train_x, train_y,
               method = "glmnet",
               tuneGrid = tuneGrid,
               trControl = train_control);



ridge.mod = glmnet(train_x, train_y, alpha=0, lambda=model$bestTune$lambda)

# train mse
ridge.pred <- predict(ridge.mod, newx = train_x )
mse <- mean((ridge.pred - train_data[, 1])^2)
print(paste("MSE of the 70% train data (12 month prediction): ", mse)) 


#test mse
ridge.pred <- predict(ridge.mod, newx = test_x )
mse <- mean((ridge.pred - test_data[, 1])^2)
print(paste("MSE of the rest 30% data (12 month prediction): ", mse)) 


#score mse
ridge.pred <- predict(ridge.mod,  newx = score_x )
mse_score <- mean((ridge.pred - data_scoreMSE[, 1])^2)
print(paste("MSE of real test data (12 month prediction): ", mse_score)) 

Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
“There were missing values in resampled performance measures.”


[1] "MSE of the 70% train data (12 month prediction):  0.000447126772261063"
[1] "MSE of the rest 30% data (12 month prediction):  0.000685415758484775"
[1] "MSE of real test data (12 month prediction):  32.8410247876629"


In [68]:
# CV to choose best lambda.    lasso

# Sliding Window Cross-Validation
train_control <- trainControl(method = "timeslice",
                              initialWindow = floor(0.7*nrow(train_x)),
                              horizon = 1,
                              skip = 1,
                              fixedWindow = TRUE,
                              allowParallel = TRUE)

#choose best lambda
tuneGrid <- expand.grid(
  alpha = 1, #lasso
  lambda = 10^seq(2, -7, length=100)
) 


model <- train(train_x, train_y,
               method = "glmnet",
               tuneGrid = tuneGrid,
               trControl = train_control);





lasso.mod = glmnet(train_x, train_y, alpha=1, lambda=model$bestTune$lambda)

# train mse
lasso.pred <- predict(lasso.mod, newx = train_x )
mse <- mean((lasso.pred - train_data[, 1])^2)
print(paste("MSE of the rest 70% train data (12 month prediction): ", mse)) 

#test mse
lasso.pred <- predict(lasso.mod, newx = test_x )
mse <- mean((lasso.pred - test_data[, 1])^2)
print(paste("MSE of the rest 30% data (12 month prediction): ", mse)) 

# score mse
lasso.pred <- predict(lasso.mod,  newx = score_x )
mse_score <- mean((lasso.pred - data_scoreMSE[, 1])^2)
print(paste("MSE of real test data (12 month prediction): ", mse_score)) 

Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
“There were missing values in resampled performance measures.”


[1] "MSE of the rest 70% train data (12 month prediction):  0.0004482906674266"
[1] "MSE of the rest 30% data (12 month prediction):  0.00129040162192826"
[1] "MSE of real test data (12 month prediction):  33.4172495393787"


## Random Forest

in this part, we use caret package to help conduct CV. Still,we use 70% train data to train RF model. <br>firstly, we apply CV method to choose best m and n.trees, and use the best tunes to train RF model. since the data is time series data, we use Sliding Window Cross-Validation<br>then use the trained model to predict test data and score data

In [24]:
# 1 month prediction for RF model

train_x = train_x_1
train_y = train_y_1
test_x = test_x_1
test_y = test_y_1
score_y = score_y_1

train_data = train_data_1
test_data = test_data_1
data_scoreMSE = data_score_1


# initialise a data.frame to store best tune M, and RMSE
rftune <- data.frame()
rftune$rmse <- c()
rftune$mtry <- c()

set.seed(1)


star = Sys.time()

# tuneGrid parameter only allowed 3 input "mtry" "splitrule" and 'min.node.size'
# thus, we write a loop to find another best tune  'n.trees'.
# we need to find best mtry and n.trees

for (i in seq(1,5)){   
train_control <- trainControl(method = "timeslice",
              initialWindow = floor(0.7*nrow(train_x)),
              horizon = 12,
              skip = 3,
              fixedWindow = TRUE,
              allowParallel = TRUE)


tuneGrid <- expand.grid(
                       mtry =floor(ncol(train_x)/seq(1,10)), # best mtry
                       splitrule= 'variance',
                       min.node.size = 5) 

model <- train(train_x, train_y,
   method = "ranger",
   tuneGrid = tuneGrid,
   trControl = train_control,
   num.trees = 100*i  # best n.trees
              );


# save the best RMSE and mtry in every loop
rftune[i,'rmse'] = min(model$results$RMSE)
rftune[i,'mtry'] = model$bestTune$mtry
}

end = Sys.time()

# cv time
print(paste("total cv time", end-star))

[1] "total cv time 1.16389551957448"


In [25]:
best_rfmodel <- ranger(x= train_x, y = train_y,  
                     data = train_data,  
                     num.trees = 100*which.min(rftune$rmse),   #best tree number
                     mtry = rftune[which.min(rftune$rmse),'mtry'],  #best mtry 
                     min.node.size = 5)
#train mse
predictions <- predict(best_rfmodel, data = train_data)
mse <- mean((predictions$predictions - train_data[, 1])^2)
print(paste("MSE of the 70% train data (1 month prediction): ", mse))


#test mse
predictions <- predict(best_rfmodel, data = test_data)
mse <- mean((predictions$predictions - test_data[, 1])^2)
print(paste("MSE of the rest 30% data (1 month prediction): ", mse))

#score mse
predictions <- predict(best_rfmodel, data = data_scoreMSE)
mse <- mean((predictions$predictions - data_scoreMSE[, 1])^2)
print(paste("MSE of real test data (1 month prediction):  ", mse))



[1] "MSE of the 70% train data (1 month prediction):  4.67984766728327e-05"
[1] "MSE of the rest 30% data (1 month prediction):  0.000665423885899809"
[1] "MSE of real test data (1 month prediction):   32.1402803761187"


In [26]:
# 3 month prediction for RF model

train_x = train_x_3
train_y = train_y_3
test_x = test_x_3
test_y = test_y_3
score_y = score_y_3

train_data = train_data_3
test_data = test_data_3
data_scoreMSE = data_score_3


# initialise a data.frame to store best tune M, and RMSE
rftune <- data.frame()
rftune$rmse <- c()
rftune$mtry <- c()

set.seed(1)

star = Sys.time()

# tuneGrid parameter only allowed 3 input "mtry" "splitrule" and 'min.node.size'
# thus, we write a loop to find another best tune  'n.trees'.
# we need to find best mtry and n.trees
for (i in seq(1,5)){
train_control <- trainControl(method = "timeslice",
              initialWindow = floor(0.7*nrow(train_x)),
              horizon = 12,
              skip = 3,
              fixedWindow = TRUE,
              allowParallel = TRUE)


tuneGrid <- expand.grid(
                       mtry =floor(ncol(train_x)/seq(1,10)),
                       splitrule= 'variance',
                       min.node.size = 5) 

model <- train(train_x, train_y,
   method = "ranger",
   tuneGrid = tuneGrid,
   trControl = train_control,
   num.trees = 100*i);



rftune[i,'rmse'] = min(model$results$RMSE)
rftune[i,'mtry'] = model$bestTune$mtry
}

end = Sys.time()

#cv time
print(paste("total cv time", end-star))

[1] "total cv time 1.15544568697611"


In [27]:
best_rfmodel <- ranger(x= train_x, y = train_y,   
                     data = train_data,  
                     num.trees = 100*which.min(rftune$rmse), #best tree number
                     mtry = rftune[which.min(rftune$rmse),'mtry'], #best mtry
                     min.node.size = 5)
#train mse
predictions <- predict(best_rfmodel, data = train_data)
mse <- mean((predictions$predictions - train_data[, 1])^2)
print(paste("MSE of the 70% train data (3 month prediction): ", mse))

#test mse
predictions <- predict(best_rfmodel, data = test_data)
mse <- mean((predictions$predictions - test_data[, 1])^2)
print(paste("MSE of the rest 30% data (3 month prediction): ", mse))

#score mse 
predictions <- predict(best_rfmodel, data = data_scoreMSE)
mse <- mean((predictions$predictions - data_scoreMSE[, 1])^2)
print(paste("MSE of real test data (3 month prediction):  ", mse))

[1] "MSE of the 70% train data (3 month prediction):  4.5714799646205e-05"
[1] "MSE of the rest 30% data (3 month prediction):  0.00114464091725681"
[1] "MSE of real test data (3 month prediction):   32.1440979979803"


In [28]:
# 12 month prediction for RF model

train_x = train_x_12
train_y = train_y_12
test_x = test_x_12
test_y = test_y_12
score_y = score_y_12

train_data = train_data_12
test_data = test_data_12
data_scoreMSE = data_score_12


# initialise a data.frame to store best tune M, and RMSE
rftune <- data.frame()
rftune$rmse <- c()
rftune$mtry <- c()

set.seed(1)

star = Sys.time()

# tuneGrid parameter only allowed 3 input: "mtry" "splitrule" and 'min.node.size'
# thus, we write a loop to find another best tune  'n.trees'.
# we need to find best mtry and n.trees
for (i in seq(1,5)){
train_control <- trainControl(method = "timeslice",
              initialWindow = floor(0.7*nrow(train_x)),
              horizon = 12,
              skip = 3,
              fixedWindow = TRUE,
              allowParallel = TRUE)


tuneGrid <- expand.grid(
                       mtry =floor(ncol(train_x)/seq(1,10)),
                       splitrule= 'variance',
                       min.node.size = 5) 

model <- train(train_x, train_y,
   method = "ranger",
   tuneGrid = tuneGrid,
   trControl = train_control,
   num.trees = 100*i);



rftune[i,'rmse'] = min(model$results$RMSE)
rftune[i,'mtry'] = model$bestTune$mtry
}

end = Sys.time()

print(paste("total cv time", end-star))

[1] "total cv time 1.10922631025314"


In [29]:

best_rfmodel <- ranger(x= train_x, y = train_y,   
                     data = train_data,  
                     num.trees = 100*which.min(rftune$rmse),  # best tree number
                     mtry = rftune[which.min(rftune$rmse),'mtry'], #best mtry
                     min.node.size = 5)
# train mse
predictions <- predict(best_rfmodel, data = train_data)
mse <- mean((predictions$predictions - train_data[, 1])^2)
print(paste("MSE of the 70% train data (12 month prediction): ", mse))

# test mse
predictions <- predict(best_rfmodel, data = test_data)
mse <- mean((predictions$predictions - test_data[, 1])^2)
print(paste("MSE of the rest 30% data (12 month prediction): ", mse))

# score mse
predictions <- predict(best_rfmodel, data = data_scoreMSE)
mse <- mean((predictions$predictions - data_scoreMSE[, 1])^2)
print(paste("MSE of real test data (12 month prediction):  ", mse))

[1] "MSE of the 70% train data (12 month prediction):  4.61976537009202e-05"
[1] "MSE of the rest 30% data (12 month prediction):  0.000906436889446865"
[1] "MSE of real test data (12 month prediction):   32.1377420462289"


## Gradient Boosting

in this part, we use caret package to help conduct CV. Still,we use 70% train data to train RF model. <br>firstly, we apply CV method to choose best depth, n.trees and shrinkage, and use the best tunes to train GBRT model. since the data is time series data, we use Sliding Window Cross-Validation<br>then use the trained model to predict test data and score data

In [36]:
# 1 month predition of gradient Boosting
train_x = train_x_1
train_y = train_y_1
test_x = test_x_1
test_y = test_y_1
score_y = score_y_1

train_data = train_data_1
test_data = test_data_1
data_scoreMSE = data_score_1

set.seed(1)


star = Sys.time()
train_control <- trainControl(method = "timeslice",
              initialWindow = floor(0.7*nrow(train_x)),
              horizon = 12,skip =3,
              fixedWindow = TRUE,
              allowParallel = TRUE)

# generate a tune grid 
gbmGrid <- expand.grid(
  interaction.depth = seq(1, 5),
  n.trees = c(50,100,200),
  shrinkage = c(0.01,0.1),
  n.minobsinnode = 10
)


model <- train(train_x, train_y,
   method = "gbm",
   tuneGrid=gbmGrid,
   trControl = train_control);

end = Sys.time()


print(paste("total cv time", end-star))
print(model$bestTune)


Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
“There were missing values in resampled performance measures.”


Iter   TrainDeviance   ValidDeviance   StepSize   Improve
     1        0.0008            -nan     0.1000    0.0001
     2        0.0007            -nan     0.1000    0.0001
     3        0.0006            -nan     0.1000    0.0001
     4        0.0006            -nan     0.1000    0.0000
     5        0.0005            -nan     0.1000    0.0000
     6        0.0005            -nan     0.1000    0.0000
     7        0.0005            -nan     0.1000    0.0000
     8        0.0005            -nan     0.1000    0.0000
     9        0.0004            -nan     0.1000    0.0000
    10        0.0004            -nan     0.1000    0.0000
    20        0.0003            -nan     0.1000    0.0000
    40        0.0003            -nan     0.1000    0.0000
    60        0.0002            -nan     0.1000    0.0000
    80        0.0002            -nan     0.1000   -0.0000
   100        0.0002            -nan     0.1000   -0.0000

[1] "total cv time 5.84427857398987"
   n.trees interaction.depth shrin

In [37]:
gbm_model <- gbm(train_y ~ ., data = train_data, distribution = "gaussian", n.trees = model$bestTune$n.trees, interaction.depth = model$bestTune$interaction.depth,shrinkage = model$bestTune$shrinkage)

# train mse
predictions <- predict(gbm_model, newdata = train_data, type = "response")
mse <- mean((predictions - train_data[, 1])^2)
print(paste("MSE of the 70% train data (1 month prediction): ", mse))

# test mse
predictions <- predict(gbm_model, newdata = test_data, type = "response")
mse <- mean((predictions - test_data[, 1])^2)
print(paste("MSE of the rest 30% data (1 month prediction): ", mse))

# score mse
predictions <- predict(gbm_model, newdata = data_scoreMSE, type = "response")
mse_score <- mean((predictions - data_scoreMSE[, 1])^2)
print(paste("MSE of real test data (1 month prediction): ", mse_score))

Using 100 trees...




[1] "MSE of the 70% train data (1 month prediction):  3.83555253806123e-06"


Using 100 trees...




[1] "MSE of the rest 30% data (1 month prediction):  0.000137858855899837"


Using 100 trees...




[1] "MSE of real test data (1 month prediction):  31.5396936753651"


In [38]:
# 3 month predition of gradient Boosting
train_x = train_x_3
train_y = train_y_3
test_x = test_x_3
test_y = test_y_3
score_y = score_y_3

train_data = train_data_3
test_data = test_data_3
data_scoreMSE = data_score_3

set.seed(3)

star = Sys.time()
train_control <- trainControl(method = "timeslice",
              initialWindow = floor(0.7*nrow(train_x)),
              horizon = 12,skip =3,
              fixedWindow = TRUE,
              allowParallel = TRUE)

#generate a tune grid
gbmGrid <- expand.grid(
  interaction.depth = seq(1, 5),
  n.trees = c(50,100,200),
  shrinkage = c(0.01,0.1),
  n.minobsinnode = 10
)


model <- train(train_x, train_y,
   method = "gbm",
   tuneGrid=gbmGrid,
   trControl = train_control);
end = Sys.time()

print(paste("total cv time", end-star))
print(model$bestTune)



Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
“There were missing values in resampled performance measures.”


Iter   TrainDeviance   ValidDeviance   StepSize   Improve
     1        0.0008            -nan     0.1000    0.0001
     2        0.0007            -nan     0.1000    0.0001
     3        0.0006            -nan     0.1000    0.0001
     4        0.0006            -nan     0.1000    0.0000
     5        0.0005            -nan     0.1000    0.0000
     6        0.0005            -nan     0.1000    0.0000
     7        0.0005            -nan     0.1000    0.0000
     8        0.0004            -nan     0.1000    0.0000
     9        0.0004            -nan     0.1000    0.0000
    10        0.0004            -nan     0.1000    0.0000
    20        0.0003            -nan     0.1000    0.0000
    40        0.0002            -nan     0.1000   -0.0000
    50        0.0002            -nan     0.1000   -0.0000

[1] "total cv time 5.85430669784546"
   n.trees interaction.depth shrinkage n.minobsinnode
22      50                 3       0.1             10


In [39]:
gbm_model <- gbm(train_y ~ ., data = train_data, distribution = "gaussian", n.trees = model$bestTune$n.trees, interaction.depth = model$bestTune$interaction.depth,shrinkage = model$bestTune$shrinkage)

# train mse
predictions <- predict(gbm_model, newdata = train_data, type = "response")
mse <- mean((predictions - train_data[, 1])^2)
print(paste("MSE of the 70% train data (3 month prediction): ", mse))

# test mse
predictions <- predict(gbm_model, newdata = test_data, type = "response")
mse <- mean((predictions - test_data[, 1])^2)
print(paste("MSE of the rest 30% data (3 month prediction): ", mse))

# score mse
predictions <- predict(gbm_model, newdata = data_scoreMSE, type = "response")
mse_score <- mean((predictions - data_scoreMSE[, 1])^2)
print(paste("MSE of real test data (3 month prediction): ", mse_score))

Using 50 trees...




[1] "MSE of the 70% train data (3 month prediction):  6.57179339492043e-06"


Using 50 trees...




[1] "MSE of the rest 30% data (3 month prediction):  0.000131966284237335"


Using 50 trees...




[1] "MSE of real test data (3 month prediction):  31.558880838485"


In [34]:
# 12 month predition of gradient Boosting
train_x = train_x_12
train_y = train_y_12
test_x = test_x_12
test_y = test_y_12
score_y = score_y_12

train_data = train_data_12
test_data = test_data_12
data_scoreMSE = data_score_12

set.seed(12)

star = Sys.time()
train_control <- trainControl(method = "timeslice",
              initialWindow = floor(0.7*nrow(train_x)),
              horizon = 12,sk
              fixedWindow = TRUE,
              allowParallel = TRUE)

#tune grid
gbmGrid <- expand.grid(
  interaction.depth = seq(1, 5),
  n.trees = c(50,100,200),
  shrinkage = c(0.01,0.1),
  n.minobsinnode = 10
)


model <- train(train_x, train_y,
   method = "gbm",
   tuneGrid=gbmGrid,
   trControl = train_control);
end = Sys.time()


print(paste("total cv time", end-star))
print(model$bestTune)



Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
“There were missing values in resampled performance measures.”


Iter   TrainDeviance   ValidDeviance   StepSize   Improve
     1        0.0008            -nan     0.1000    0.0001
     2        0.0007            -nan     0.1000    0.0001
     3        0.0007            -nan     0.1000    0.0000
     4        0.0007            -nan     0.1000    0.0000
     5        0.0006            -nan     0.1000    0.0000
     6        0.0006            -nan     0.1000    0.0000
     7        0.0006            -nan     0.1000    0.0000
     8        0.0006            -nan     0.1000    0.0000
     9        0.0005            -nan     0.1000    0.0000
    10        0.0005            -nan     0.1000    0.0000
    20        0.0004            -nan     0.1000    0.0000
    40        0.0003            -nan     0.1000    0.0000
    60        0.0003            -nan     0.1000   -0.0000
    80        0.0003            -nan     0.1000   -0.0000
   100        0.0003            -nan     0.1000   -0.0000

[1] "total cv time 22.9334223270416"
   n.trees interaction.depth shrin

In [35]:
gbm_model <- gbm(train_y ~ ., data = train_data, distribution = "gaussian", n.trees = model$bestTune$n.trees, interaction.depth = model$bestTune$interaction.depth,shrinkage = model$bestTune$shrinkage)

# train mse
predictions <- predict(gbm_model, newdata = train_data, type = "response")
mse <- mean((predictions - train_data[, 1])^2)
print(paste("MSE of the 70% train data (12 month prediction): ", mse))

#test mse
predictions <- predict(gbm_model, newdata = test_data, type = "response")
mse <- mean((predictions - test_data[, 1])^2)
print(paste("MSE of the rest 30% data (12 month prediction): ", mse))

#score mse
predictions <- predict(gbm_model, newdata = data_scoreMSE, type = "response")
mse_score <- mean((predictions - data_scoreMSE[, 1])^2)
print(paste("MSE of real test data (12 month prediction): ", mse_score))

Using 100 trees...




[1] "MSE of the 70% train data (12 month prediction):  6.60612864759031e-06"


Using 100 trees...




[1] "MSE of the rest 30% data (12 month prediction):  0.000129689456290248"


Using 100 trees...




[1] "MSE of real test data (12 month prediction):  31.541134467644"
